## Install OpenpyXL package

In [2]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 769.8 kB/s eta 0:00:0000:010:01m


### Importing Packages

In [3]:
import pandas as pd
import numpy as np
import os
import csv
import glob
import shutil

from openpyxl import load_workbook

In [4]:
def getMonthNumber(Month):
    month_map = {'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 
                    'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12'}
    return month_map.get(Month)

def getDate(row):
    if row['year'] is None:
        return row[0]
    else:
        return row['year'] + '-' + row['month'] + '-01'

def loadExcel(inputFullFile):
    sheetName = r'Products'
    wb = load_workbook(inputFullFile, data_only=True)
    ws = wb[sheetName]

    data = ws.values
    df = pd.DataFrame(data)
    return ws, df

def getColorsAndComments(ws):
    colors = []
    comments = []
    for row in ws.iter_rows():
        row_colors = []
        row_comments = []
        for cell in row:
            row_colors.append(cell.fill.start_color.index)
            row_comments.append(cell.comment)
        colors.append(row_colors)
        comments.append(row_comments)
    
    df_clrs = pd.DataFrame(colors)
    df_cmms = pd.DataFrame(comments)
    return df_clrs, df_cmms

def cleanData(RowRegions, df, df_colors, df_comments):
    dx          = df[RowRegions:RowRegions+1].T                   #Get Regions row
    dx.columns  = {'Regions'} 
    col_index   = list(dx[dx['Regions'] != 'Average'].index)      #Get Index List without Average columns
    df          = df.filter(items = col_index, axis=1)            #DataFrame without Average Columns
    df_colors   = df_colors.filter(items = col_index, axis=1) 
    df_comments = df_comments.filter(items = col_index, axis=1) 
   
    MaxRows = df[df.isnull()[0]].index[0]                         #Remove Quarter/Year Summary
    df = df[:MaxRows]
    df_colors = df_colors[:MaxRows]
    df_comments = df_comments[:MaxRows]
                                                                 #New Column with year and month number equivalent
    df['year'] = df[0].apply(lambda x: str(x) if isinstance(x, int) else None).ffill()
    df['month'] = df[0].apply(getMonthNumber)

    df = df.astype({0:str})                                      #Convert first column to String
    row_index = list(df[df[0]!=df['year']].index)                #Get Index List without Totals by year in rows
    df = df.filter(items = row_index, axis=0)                    #DataFrame without Average Columns 
    df_colors = df_colors.filter(items = row_index, axis=0)
    df_comments = df_comments.filter(items = row_index, axis=0)

    df[0] = df.apply(getDate, axis=1)                            #Assign to first column dates or tittles

    df.columns = range(df.columns.size)                          #Reset Index rows to consecutive numbers (deleted years rows)
    df.reset_index(drop=True)                                    #Reset Columns names to consecutive numbers

    df_colors.columns = range(df_colors.columns.size)
    df_colors.reset_index(drop=True)

    df_comments.columns = range(df_comments.columns.size)
    df_comments.reset_index(drop=True)
    
    return df.iloc[:, :-2],df_colors,df_comments                #Remove Year and Month columns in all dataframes

def getProducts(RowStartData, MaxCols, df):
    df_products = df.iloc[:RowStartData, 0:MaxCols].T
    df_products.columns = df_products.iloc[0]                    #Assign columns names
    df_products = df_products.iloc[1:,:]                         #Remove first row
    df_products = df_products.astype('string')                   #Change data type to String
    df_products['Id'] = df_products.index
    return df_products

def getData(RowDataStart, reported_period, df, df_colors, df_comments):
    ForecastColor = 9                                           #Excel color coding for Green Light
    
    df_data = df.iloc[RowDataStart:, :].T                       #Get data and pivot dates to columns
    df_clrs = df_colors.iloc[RowDataStart:, :].T                #Get colors
    df_cmms = df_comments.iloc[RowDataStart:, :].T              #Get comments
    
    df_data.columns = df_data.iloc[0]                           #Column name with dates
    df_clrs.columns = df_data.iloc[0]                           #Column name with dates
    df_cmms.columns = df_data.iloc[0]                           #Column name with dates
    
    df_data = df_data.iloc[1:,:]                                #Remove first row with dates
    df_clrs = df_clrs.iloc[1:,:]                                #Remove first row with dates
    df_cmms = df_cmms.iloc[1:,:]                                #Remove first row with dates
    
    df_data = df_data.melt()                                    #Pivot by period
    df_clrs = df_clrs.melt()                                    #Pivot by period
    df_cmms = df_cmms.melt().fillna("")                         #Pivot by period and change None to Empty

    df_data.columns = ['Period','Value']
    df_clrs.columns = ['CLR Period','Color']
    df_cmms.columns = ['CMMS Period','Comments']

    df_data = pd.concat([df_data,df_clrs['Color'],df_cmms['Comments']],axis=1)     #Adding Colors and Comments
        
    df_data['Id'] = df_data.groupby(df_data['Period']).cumcount() + 1              #Add incremental count by period starting in 1
    df_data['View'] = np.where((df_data['Color']==ForecastColor),'Forecast','Actual') 
    
    df_data['Reported_period'] = reported_period                                   #Reported Period from the name of the file
    return df_data
   

## Processing Excel Files:

#### Change the __inputPath___ and ___outputPath___ variables with the correct address:

In [5]:
print('[Starting ETL...]')
inputPath = r'/home/csaavedra/tc24/1062/demo1/input/'
outputPath = r'/home/csaavedra/tc24/1062/demo1/temp/'

files = glob.glob(os.path.join(inputPath,'*.xlsx'), recursive=False)

for f in files:
    print('Reading file: {0}'.format(f))
    reported_period = f[-11:-7] + '-' + f[-7:-5]
    (ws, df) = loadExcel(f)
    (df_clrs, df_cmmnts) = getColorsAndComments(ws)

    RowDataStart = 4           #First Line with Data beginning from zero    
    MaxCols  = df.shape[1]
    
    (df, df_clrs, df_cmmnts) = cleanData(RowDataStart-1,df, df_clrs, df_cmmnts)
    df_prds  = getProducts(RowDataStart, MaxCols, df)

    df_data  = getData(RowDataStart, reported_period, df, df_clrs, df_cmmnts)
    df_final = pd.merge(left=df_prds, right=df_data, how='inner', on='Id')

    filename = f.split('/')[-1][:-5] + '.csv'
    output_file = os.path.join(outputPath, filename)
       
    print('Saving file:  {0}'.format(output_file))
    df_final.to_csv(output_file, index=False, sep=',')
    print("[Done]")
    
print("[End]")

[Starting ETL...]
Reading file: /home/csaavedra/tc24/1062/demo1/input/Actual and Forecast Consolidated 202401.xlsx
Saving file:  /home/csaavedra/tc24/1062/demo1/temp/Actual and Forecast Consolidated 202401.csv
[Done]
Reading file: /home/csaavedra/tc24/1062/demo1/input/Actual and Forecast Consolidated 202402.xlsx
Saving file:  /home/csaavedra/tc24/1062/demo1/temp/Actual and Forecast Consolidated 202402.csv
[Done]
[End]


### How looks the DataFrame?

In [8]:
df_final.tail(12)

,Product,UOM,Category,Region,Id,Period,Value,Color,Comments,View,Reported_period
756,Wheat,Bushel,Staple food,North America,16,2024-01-01,13.8,00000000,,Actual,2024-02
757,Wheat,Bushel,Staple food,North America,16,2024-02-01,13.2,00000000,,Actual,2024-02
758,Wheat,Bushel,Staple food,North America,16,2024-03-01,14.2,9,,Forecast,2024-02
759,Wheat,Bushel,Staple food,North America,16,2024-04-01,14.4,9,,Forecast,2024-02
760,Wheat,Bushel,Staple food,North America,16,2024-05-01,14.6,9,,Forecast,2024-02
761,Wheat,Bushel,Staple food,North America,16,2024-06-01,14.8,9,,Forecast,2024-02
762,Wheat,Bushel,Staple food,North America,16,2024-07-01,15,9,,Forecast,2024-02
763,Wheat,Bushel,Staple food,North America,16,2024-08-01,15.2,9,,Forecast,2024-02
764,Wheat,Bushel,Staple food,North America,16,2024-09-01,15.4,9,,Forecast,2024-02
765,Wheat,Bushel,Staple food,North America,16,2024-10-01,15.6,9,,Forecast,2024-02


### How looks the Color DataFrame?

In [7]:
from IPython.core.display import HTML
display(HTML(df_clrs.to_html()))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
2,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
3,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
5,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
6,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
7,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
8,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
9,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
10,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000
